# 웹에 머신러닝 모델 내장
### 분류, 실시간으로 학습하는 머신러닝 모델의 웹 적용
1. 머신러닝 모델 현재 훈련 상태 저장하기
2. SQLite DB 사용하여 데이터 저장하기
3. Flask
4. 공개 웹 서버에 머신 러닝 애플리케이션 배포하기

### 1. 머신러닝 모델 현재 훈련 상태 저장하기

In [ ]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

'''
stop : model
clf : model
'''
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol = 4)
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol = 4)

### 1-1. 저장된 머신러닝 모델을 불러와 테스트까지 적용

In [3]:
import pickle
import re
import os
from vectorizer import vect
'''
위의 vectorizer.py의 hashingvectorizer는 학습과정이 없기 때문에 직렬화할 필요가 없다.
'''

clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))

In [9]:
import numpy as np
label = {0:'양성', 1:'음성'}

example = ['I love this movie']
X = vect.transform(example)
print('예측:', label[clf.predict(X)[0]])
print('확률: ', round(np.max(clf.predict_proba(X))*100, 2))

예측: 음성
확률:  81.97


### 2-1 sqlite db 설정

In [10]:
import sqlite3

conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('DROP TABLE IF EXISTS review_db')
c.execute('CREATE TABLE review_db'\
          ' (review TEXT, sentiment INTEGER, date TEXT)')

example1 = 'I love this movie'
c.execute("INSERT INTO review_db"\
          " (review, sentiment, date) VALUES"\
          " (?, ?, DATETIME('now'))", (example1, 1))

example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db"\
          " (review, sentiment, date) VALUES"\
          " (?, ?, DATETIME('now'))", (example2, 0))

conn.commit()
conn.close()

In [11]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute("SELECT * FROM review_db WHERE date"\
          " BETWEEN '2017-01-01 00:00:00' AND DATETIME('now')")
results = c.fetchall()
conn.close()
print(results)

[('I love this movie', 1, '2021-02-25 04:35:52'), ('I disliked this movie', 0, '2021-02-25 04:35:52')]
